## Applying Hierarchical LDA Model

Hierarchical Latent Dirichlet Allocation (hLDA) addresses the problem of learning topic hierarchies from data. The model relies on a non-parametric prior called the nested Chinese restaurant process, which allows for arbitrarily large branching factors and readily accommodates growing data collections. The hLDA model combines this prior with a likelihood that is based on a hierarchical variant of latent Dirichlet allocation.

Idea inspired from [Hierarchical Topic Models and the Nested Chinese Restaurant Process](http://www.cs.columbia.edu/~blei/papers/BleiGriffithsJordanTenenbaum2003.pdf)

<font color="blue"/>

### dsp:
  * &#x1f642; Wow! That is something extra. Great!
  * To reference a scientific publication, you should give more complete data, e.g. for the 2010 version of the document: "David M. Blei, Thomas L. Griffiths, and Michael I. Jordan. 2010. The nested chinese restaurant process and bayesian nonparametric inference of topic hierarchies. J. ACM 57, 2, Article 7 (February 2010), 30 pages. DOI: https://doi.org/10.1145/1667053.1667056"

#### Hierarchical LDA Model example
![Hierarchical LDA Model example](Hierarchical_LDA_Model_example.png)

In [ ]:
from gensim import corpora, models, similarities
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim
import os
import pickle
import pandas as pd
import numpy as np
import warnings
pyLDAvis.enable_notebook()

<font color="blue"/>

### dsp:
  * Sorting and adding empty lines might make this import list more accessible.

#### 1. Apply the initial LDA Model -- In order to group those documents which are seemed to belong to the same topic
- params:
    - folder_path: cleand preprocessed data folder
    - non_german_file_path: a list of all English files
    - topic_num: topics numbers to choose while training the model
    - document_limit: restrict the number of documents to process
- variables:
    - gensim_dictionary: [Dictionay](https://radimrehurek.com/gensim/corpora/dictionary.html) type, defined by gesim library 
    - corpus: doc2bow, tokenized and nomalized list
    - lda: LDA model after training with given dataset

<font color="blue"/>

### dsp:
  * "are seemed" ~> "seem"
  * What you call a "path" here you have been calling a "folder" before.

In [ ]:
def initialLDAModel(folder_path, non_german_file_path, topic_num=50, document_limit=1000):
    
    #get a list of non_german files
    non_german_files = []
    with open(non_german_file_path, 'r') as fr:
        for line in fr:
            non_german_files.append(line.strip())
    
    dictionary = []
    filenames = []
    
    for root, dirs, files in os.walk(folder_path):
        for f in files:
            if f[:-11] not in non_german_files:
                document_limit -= 1
                try:
                    with open(root+'/'+f, 'rb') as fr:
                        filenames.append(f)
                        document_tokens = pickle.load(fr)
                        dictionary.append(document_tokens)
                except:
                    print('Error while processing: ', f)
            
            if document_limit == 0:
                break
    
    gensim_dictionary = Dictionary(dictionary)
    corpus = [gensim_dictionary.doc2bow(text) for text in dictionary]
    lda = LdaModel(corpus, num_topics=topic_num, id2word=gensim_dictionary, iterations=200)
    topics = lda.show_topics(num_topics=-1, num_words=20)
    
    doc_pos = 0
    mat = np.zeros((len(filenames), topic_num))

    for doc in corpus:
        vector = lda[doc] # get topic probability distribution for a document
        for element in vector:
            mat[doc_pos][element[0]] = element[1]
        doc_pos += 1
        
    df = pd.DataFrame(mat, index=filenames, columns=range(0,topic_num))
    
    return df

<font color="blue"/>

### dsp:
  * The first half of this function could be a function in its own right. I think I have seen similar code in other notebooks as well.

Please check the [Folder Structure](01-Convert_JSON_to_TXT.ipynb) if there is running error.

**non_german_files.txt** should be located in the same folder, if not, please use the absolute path to locaate that file.

In [ ]:
%%time
warnings.filterwarnings("ignore")
#please modify the path
# non_german_file_path = '/home/bit/ma0/LabShare/data/non_german_files.txt'
# processed_doc_path = '/home/bit/ma0/LabShare/data/chui_ma/spacy_corpus/'

#relative path
non_german_file_path = './non_german_files.txt'
processed_doc_path = '../spacy_corpus/'
df = initialLDAModel(processed_doc_path, non_german_file_path, 50, 500)

<font color="blue"/>

### dsp:
  * Do you know, which warning you suppress? Did you check whether it is OK to ignore them? Are there no other warnings?

#### Group topic related documents by assuming that belong to the same topic with the probability larget than 0.3
- params: 
    - df: pandas DataFrame, each column represents one topic, and each row indicates the probability of that document belongs to this topic
- variables: 
    - topic_related_documents: list, contain group of doucment names which seem to belong to the same topic

In [5]:
def get_Topic_Related_Doc(df):  
    
    topic_related_documents = []
    
    for column in range(df.shape[1]):
        row_count = 0
        for row in range(df.shape[0]):
            if df.iat[row, column] > 0.3:
                row_count += 1
        topic_related_documents.append(df.nlargest(row_count, column).index)
    
    return topic_related_documents

#### 2. Apply the HierachicalLDA LDA Model 
- params:
    - folder_path: cleand preprocessed data folder
    - non_german_file_path: a list of all English files
    - topic_num: topics numbers to choose while training the model
    - document_limit: restrict the number of documents to process
- variables:
    - gensim_dictionary: [Dictionay](https://radimrehurek.com/gensim/corpora/dictionary.html) type, defined by gesim library 
    - corpus: doc2bow, tokenized and nomalized list
    - lda: LDA model after training with given dataset

In [ ]:
def hierachicalLDA(folder_path, non_german_file_path, topic_related_documents, topic_num=5, document_limit=1000):
    #get a list of non_german files
    non_german_files = []
    with open(non_german_file_path, 'r') as fr:
        for line in fr:
            non_german_files.append(line.strip())
            
    dictionary = []
    filenames = []
    for root, dirs, files in os.walk(folder_path):
        for f in sorted(files):
            if f in topic_related_documents:
                if f[:-11] not in non_german_files:
                    with open(root+'/'+f, 'rb') as fr:
                        filenames.append(f)
                        document_tokens = pickle.load(fr)
                        dictionary.append(document_tokens)

    gensim_dictionary = Dictionary(dictionary)
    corpus = [gensim_dictionary.doc2bow(text) for text in dictionary]
    lda = models.ldamodel.LdaModel(corpus, num_topics=topic_num, id2word=gensim_dictionary, iterations=200)
    topics = lda.show_topics(num_topics=-1, num_words=20)
    
    return lda, corpus, gensim_dictionary

<font color="blue"/>

### dsp:
  * This is probably the special part of this notebook.
  * You did not document the parameter `topic_related_documents`.
  * Does this parameter realized the "hierarchical" aspect of your "hierarchical LDA"?
  * You do not seem to pass a document list containing only "topic related" documents, do you?

In [7]:
docs = get_Topic_Related_Doc(df)

#### Visualize the topic model by pyLDAvis library
- params: 
    - folder_path: cleand preprocessed data folder
    - docs: group of documents after initial LDA Model
    - topic_num: topics number to choose

In [ ]:
def visualization(data_path, docs, topic_num):
    
    #only visualize those topics with more than 5 documents
    if len(docs) > 5:
        lda, corpus, dictionary = hierachicalLDA(data_path, non_german_file_path, docs, topic_num)
        return pyLDAvis.gensim.prepare(lda, corpus, dictionary)
    
    else:
        print('Cannot visualize, Too less documents')

<font color="blue"/>

### dsp:
  * "Too less documents" ~> "Too few documents"

Give a simple example, apply the Hierarchical LDA on one topic that has the maximum related documents.

In [9]:
# visualization(processed_doc_path, docs[6].tolist(), 3)
doc_example = max(docs, key=len)
visualization(processed_doc_path, doc_example.tolist(), 3)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.005745  0.006247       1        1  54.019630
0     -0.004192 -0.006934       2        1  42.418159
2      0.009937  0.000687       3        1   3.562213, topic_info=     Category        Freq             Term       Total  loglift  logprob
term                                                                    
1088  Default  780.000000          vorjahr  780.000000  30.0000  30.0000
307   Default  708.000000         ergebnis  708.000000  29.0000  29.0000
481   Default  911.000000             höhe  911.000000  28.0000  28.0000
806   Default  935.000000          quartal  935.000000  27.0000  27.0000
567   Default  678.000000          konzern  678.000000  26.0000  26.0000
1024  Default  647.000000      unternehmen  647.000000  25.0000  25.0000
857   Default  588.000000           risiko  588.000000  24.0000  24.0000
299   Default  698.000000      entwicklung  698.000000  23.0000  23.0000
403   Default  607.000000    geschäftsjahr  607.000000  22.0000  22.0000
658   Default  444.000000            markt  444.000000  21.0000  21.0000
28    Default  451.000000            aktie  451.000000  20.0000  20.0000
482   Default  498.000000             ifrs  498.000000  19.0000  19.0000
535   Default  429.000000             juni  429.000000  18.0000  18.0000
1134  Default  394.000000      wesentliche  394.000000  17.0000  17.0000
904   Default  746.000000          segment  746.000000  16.0000  16.0000
437   Default  532.000000           gruppe  532.000000  15.0000  15.0000
1032  Default  310.000000  verbindlichkeit  310.000000  14.0000  14.0000
809   Default  414.000000           rahmen  414.000000  13.0000  13.0000
603   Default  401.000000            kunde  401.000000  12.0000  12.0000
1060  Default  450.000000   vermögenswerte  450.000000  11.0000  11.0000
233   Default  364.000000         dezember  364.000000  10.0000  10.0000
1011  Default  428.000000           umsatz  428.000000   9.0000   9.0000
414   Default  333.000000     gesellschaft  333.000000   8.0000   8.0000
81    Default  341.000000     aufsichtsrat  341.000000   7.0000   7.0000
63    Default  442.000000           anteil  442.000000   6.0000   6.0000
695   Default  858.000000        milliarde  858.000000   5.0000   5.0000
290   Default  281.000000            enden  281.000000   4.0000   4.0000
1100  Default  298.000000        vorstehen  298.000000   3.0000   3.0000
884   Default  337.000000         rückgang  337.000000   2.0000   2.0000
84    Default  304.000000       aufwendung  304.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
567    Topic3   23.919571          konzern  678.706909  -0.0107  -5.0359
857    Topic3   21.089596           risiko  588.603760   0.0058  -5.1618
172    Topic3    7.583028        bewertung  162.062363   0.2727  -6.1846
535    Topic3   16.098795             juni  429.410217   0.0511  -5.4318
1024   Topic3   22.085148      unternehmen  647.049072  -0.0427  -5.1157
481    Topic3   28.638552             höhe  911.188660  -0.1252  -4.8558
482    Topic3   17.360649             ifrs  498.013428  -0.0216  -5.3564
403    Topic3   20.100628    geschäftsjahr  607.985291  -0.0746  -5.2098
1134   Topic3   14.395544      wesentliche  394.908722   0.0231  -5.5436
806    Topic3   27.317249          quartal  935.467773  -0.1987  -4.9030
1883   Topic3    8.976398         leistung  217.974380   0.1450  -6.0160
299    Topic3   21.102028      entwicklung  698.729675  -0.1651  -5.1612
127    Topic3    5.481580        bedeutung  113.034927   0.3085  -6.5092
233    Topic3   12.702113         dezember  364.452179  -0.0218  -5.6688
290    Topic3   10.513488            enden  281.930481   0.0458  -5.8579
414    Topic3   11.770217     gesellschaft  333.628326  -0.0097  -5.7450
603    Topic3   13.203284            kunde  401.209412  -0.0792  -5.6301
81     Topic3   11.776730     aufsich

<font color="blue"/>

### dsp:
  * Interesting experiment. 
  * I have still to study the literature.
  * From the code I can not understand how the "hierarchical" aspect is meant to work and I suspect you do not understand it either, or do you?